# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 17 2022 4:27PM,237484,16,ACG-2102490101,ACG North America LLC,Released
1,May 17 2022 4:25PM,237483,19,ACG-2102490106,ACG North America LLC,Released
2,May 17 2022 4:25PM,237483,19,ACG-2102490107,ACG North America LLC,Released
3,May 17 2022 4:25PM,237478,10,0085673152,ISDIN Corporation,Released
4,May 17 2022 4:25PM,237478,10,0085673269,ISDIN Corporation,Released
5,May 17 2022 4:25PM,237478,10,0085673277,ISDIN Corporation,Released
6,May 17 2022 4:25PM,237478,10,0085673280,ISDIN Corporation,Released
7,May 17 2022 4:25PM,237478,10,0085673285,ISDIN Corporation,Released
8,May 17 2022 4:25PM,237478,10,0085673315,ISDIN Corporation,Released
9,May 17 2022 4:25PM,237478,10,0085673313,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
60,237477,Executing,7
61,237477,Released,15
62,237478,Released,45
63,237483,Released,2
64,237484,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237476,NaN,NaN,10.0
237477,1.0,7.0,15.0
237478,NaN,NaN,45.0
237483,NaN,NaN,2.0
237484,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0.0,1.0,0.0
237382,0.0,0.0,1.0
237385,0.0,0.0,1.0
237386,0.0,0.0,1.0
237388,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0,1,0
237382,0,0,1
237385,0,0,1
237386,0,0,1
237388,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,0,1,0
1,237382,0,0,1
2,237385,0,0,1
3,237386,0,0,1
4,237388,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,,1,
1,237382,,,1
2,237385,,,1
3,237386,,,1
4,237388,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 17 2022 4:27PM,237484,16,ACG North America LLC
1,May 17 2022 4:25PM,237483,19,ACG North America LLC
3,May 17 2022 4:25PM,237478,10,ISDIN Corporation
48,May 17 2022 4:08PM,237477,10,ISDIN Corporation
71,May 17 2022 4:02PM,237476,10,ISDIN Corporation
81,May 17 2022 3:58PM,237474,16,Sartorius Stedim Filters Inc.
82,May 17 2022 3:52PM,237449,18,"Blinc International, Inc.-Blinc-ecom"
83,May 17 2022 3:52PM,237450,18,"Blinc International, Inc.-Blinc-ecom"
84,May 17 2022 3:52PM,237451,18,"Blinc International, Inc.-Blinc-ecom"
85,May 17 2022 3:51PM,237452,18,"Blinc International, Inc.-Blinc-ecom"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 17 2022 4:27PM,237484,16,ACG North America LLC,,,1
1,May 17 2022 4:25PM,237483,19,ACG North America LLC,,,2
2,May 17 2022 4:25PM,237478,10,ISDIN Corporation,,,45
3,May 17 2022 4:08PM,237477,10,ISDIN Corporation,1,7,15
4,May 17 2022 4:02PM,237476,10,ISDIN Corporation,,,10
5,May 17 2022 3:58PM,237474,16,Sartorius Stedim Filters Inc.,,,1
6,May 17 2022 3:52PM,237449,18,"Blinc International, Inc.-Blinc-ecom",,,1
7,May 17 2022 3:52PM,237450,18,"Blinc International, Inc.-Blinc-ecom",,,1
8,May 17 2022 3:52PM,237451,18,"Blinc International, Inc.-Blinc-ecom",,,1
9,May 17 2022 3:51PM,237452,18,"Blinc International, Inc.-Blinc-ecom",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 4:27PM,237484,16,ACG North America LLC,1,,
1,May 17 2022 4:25PM,237483,19,ACG North America LLC,2,,
2,May 17 2022 4:25PM,237478,10,ISDIN Corporation,45,,
3,May 17 2022 4:08PM,237477,10,ISDIN Corporation,15,7,1
4,May 17 2022 4:02PM,237476,10,ISDIN Corporation,10,,
5,May 17 2022 3:58PM,237474,16,Sartorius Stedim Filters Inc.,1,,
6,May 17 2022 3:52PM,237449,18,"Blinc International, Inc.-Blinc-ecom",1,,
7,May 17 2022 3:52PM,237450,18,"Blinc International, Inc.-Blinc-ecom",1,,
8,May 17 2022 3:52PM,237451,18,"Blinc International, Inc.-Blinc-ecom",1,,
9,May 17 2022 3:51PM,237452,18,"Blinc International, Inc.-Blinc-ecom",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 4:27PM,237484,16,ACG North America LLC,1,,
1,May 17 2022 4:25PM,237483,19,ACG North America LLC,2,,
2,May 17 2022 4:25PM,237478,10,ISDIN Corporation,45,,
3,May 17 2022 4:08PM,237477,10,ISDIN Corporation,15,7,1
4,May 17 2022 4:02PM,237476,10,ISDIN Corporation,10,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 4:27PM,237484,16,ACG North America LLC,1.0,NaN,NaN
1,May 17 2022 4:25PM,237483,19,ACG North America LLC,2.0,NaN,NaN
2,May 17 2022 4:25PM,237478,10,ISDIN Corporation,45.0,NaN,NaN
3,May 17 2022 4:08PM,237477,10,ISDIN Corporation,15.0,7.0,1.0
4,May 17 2022 4:02PM,237476,10,ISDIN Corporation,10.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 4:27PM,237484,16,ACG North America LLC,1.0,0.0,0.0
1,May 17 2022 4:25PM,237483,19,ACG North America LLC,2.0,0.0,0.0
2,May 17 2022 4:25PM,237478,10,ISDIN Corporation,45.0,0.0,0.0
3,May 17 2022 4:08PM,237477,10,ISDIN Corporation,15.0,7.0,1.0
4,May 17 2022 4:02PM,237476,10,ISDIN Corporation,10.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2374368,75.0,20.0,34.0
15,237441,0.0,1.0,0.0
16,949731,4.0,0.0,0.0
18,5698565,25.0,1.0,0.0
19,949794,23.0,7.0,0.0
20,3086418,74.0,0.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2374368,75.0,20.0,34.0
1,15,237441,0.0,1.0,0.0
2,16,949731,4.0,0.0,0.0
3,18,5698565,25.0,1.0,0.0
4,19,949794,23.0,7.0,0.0
5,20,3086418,74.0,0.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,75.0,20.0,34.0
1,15,0.0,1.0,0.0
2,16,4.0,0.0,0.0
3,18,25.0,1.0,0.0
4,19,23.0,7.0,0.0
5,20,74.0,0.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,75.0
1,15,Released,0.0
2,16,Released,4.0
3,18,Released,25.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20
Status,,,,,,
Completed,34.0,0.0,0.0,0.0,0.0,2.0
Executing,20.0,1.0,0.0,1.0,7.0,0.0
Released,75.0,0.0,4.0,25.0,23.0,74.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20
0,Completed,34.0,0.0,0.0,0.0,0.0,2.0
1,Executing,20.0,1.0,0.0,1.0,7.0,0.0
2,Released,75.0,0.0,4.0,25.0,23.0,74.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20
0,Completed,34.0,0.0,0.0,0.0,0.0,2.0
1,Executing,20.0,1.0,0.0,1.0,7.0,0.0
2,Released,75.0,0.0,4.0,25.0,23.0,74.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()